In [1]:
#pip install wget, mysql-connector-python

In [2]:
import requests
from bs4 import BeautifulSoup

## Setting Database MySQL Python

In [3]:
import mysql.connector
database_name = "crawling_database"

In [4]:
mydb = mysql.connector.connect(
  host="dbcrawling",
  user="root",
  password="root",
)
mycursor = mydb.cursor()

`delete when need it`

In [5]:
# mycursor.execute("DROP DATABASE crawling_data")

`create database`

In [6]:
mycursor.execute("CREATE DATABASE IF NOT EXISTS "+database_name)
mydb = mysql.connector.connect(
  host="dbcrawling",
  user="root",
  password="root",
  database= database_name
)
mycursor = mydb.cursor()

`create table`

In [7]:
mycursor.execute("CREATE TABLE IF NOT EXISTS crawling_link (id int NOT NULL AUTO_INCREMENT, link varchar(255), status VARCHAR(255), PRIMARY KEY (id))")
mycursor.execute("CREATE TABLE IF NOT EXISTS crawling_detail (id int NOT NULL AUTO_INCREMENT,nomor varchar(255),tingkat_proses varchar(255),klasifikasi varchar(255),kata_kunci varchar(255),tahun varchar(255),tanggal_register varchar(255),lembaga_peradilan varchar(255),jenis_lembaga_peradilan varchar(255),hakim_ketua varchar(255),hakim_anggota varchar(255),panitera varchar(255),amar varchar(255),amar_lainnya varchar(255),catatan_amar text,tanggal_musyawarah varchar(255),tanggal_dibacakan varchar(255),kaidah varchar(255),abstrak varchar(255),source_url text,nama_file text,url_file text,crawling_file text, status_download varchar(255), PRIMARY KEY (id))")

`All SQL Language`

In [8]:
sql_insert_data_crawling = "INSERT INTO crawling_link (link, status) VALUES (%s, %s)"

## Get Information Landing Page

In [9]:
landingpage = requests.get("https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=putusan&cat=&jd=&tp=&court=&t_put=&t_reg=&t_upl=&t_pr=")
soup_landingpage = BeautifulSoup(landingpage.text, 'html.parser')

## Get Last Page Number

In [10]:
pagination = soup_landingpage.find_all("a",{"class":"page-link"})
halaman = list(filter(lambda _ : _.text == "Last",pagination))[-1]
total_halaman = halaman.get('href')
num_total_halaman = int(total_halaman.split("=")[-1])
num_total_halaman

499

## Get Item Post List

In [11]:
def get_post_list(num_page,num_total_halaman=num_total_halaman):
    url_page = "https://putusan3.mahkamahagung.go.id/search.html?jenis_doc=putusan&page="+str(num_page)
    new_landingpage = requests.get(url_page)
    soup_landingpage = BeautifulSoup(new_landingpage.text, 'html.parser')
    kolom_hasil = soup_landingpage.find("div",{"class":"postcontent nobottommargin col_three_fifth col_last"})
    kasus = kolom_hasil.find_all("div", {"class": "spost clearfix"})
    for per_kasus in kasus:
        ini_link = list(filter(lambda _ : len(_.find_all("a")) > 0 ,per_kasus.find_all("strong")))
        output_link = ini_link[0].find("a").get("href")
        val_insert_data_crawling = (output_link, "belum")
        mycursor.execute(sql_insert_data_crawling, val_insert_data_crawling)
        mydb.commit()
    return(len(kasus),url_page)

In [12]:
num_awal_halaman = 1
total_case_data = 0
for _per in range(num_awal_halaman, num_total_halaman):
    total_case,url = get_post_list(_per)
    total_case_data += total_case
    print("Processing "+str(num_awal_halaman)+"/"+str(num_total_halaman)+" | Get "+str(total_case)+" data | Total Data "+str(total_case_data)+" data |"+url)
    num_awal_halaman+=1
    # break
print("Done Processing All Data")
    

Processing 1/499 | Get 20 data | Total Data 20 data |https://putusan3.mahkamahagung.go.id/search.html?jenis_doc=putusan&page=1
Done Processing All Data


## Get Kasus Per Link

In [13]:
def extract_page_detail(id_row, url_page):
    try:
        new_landingpage_kasus = requests.get(url_page)
        soup_landingpage_kasus = BeautifulSoup(new_landingpage_kasus.text, 'html.parser')
        kolom_hasil_detail = soup_landingpage_kasus.find("div",{"class":"tab-container"})
        kolom_hasil_detail_link = soup_landingpage_kasus.find("div",{"class":"col-sm-4 nobottommargin col_last"})
        kolom_kanan_hasil_link = kolom_hasil_detail_link.find("ul",{"class":"portfolio-meta nobottommargin"}).find_all("li")[-1].find("a")
        kasus_details = kolom_hasil_detail.find_all("tr")[1:]
        _all = {}
        _allvalue = []
        for _ in kasus_details:
            _data = _.find_all("td")
            _kolom,_value = _data[0].text,_data[1].text
            _all[_kolom.replace("\t","").replace("\n","").replace("\r","").lower().replace(" ","_")] = _value.replace("\t","").replace("\n","").replace("\r","")
            _allvalue.append(_value.replace("\t","").replace("\n","").replace("\r",""))
        _allvalue.append(kolom_kanan_hasil_link.text)
        _allvalue.append(kolom_kanan_hasil_link.get("href"))
        _all['source_url'] = url_page
        _all['nama_file'] = kolom_kanan_hasil_link.text
        _all['url_file'] = kolom_kanan_hasil_link.get("href")
        _all['status_download'] = "belum"

        nama_kolom = list(_all.keys())
        values_kolom = list(_all.values())
        panjang_s = ['%s' for i in range(len(nama_kolom))]

        sql_insert_data_crawling_detail = "INSERT INTO crawling_detail ("+','.join(nama_kolom)+") VALUES ("+','.join(panjang_s)+")";
        # sql_insert_data_crawling_detail = "INSERT INTO crawling_detail (nomor,tingkat_proses,klasifikasi,kata_kunci,tahun,tanggal_register,lembaga_peradilan,jenis_lembaga_peradilan,hakim_ketua,hakim_anggota,panitera,amar,amar_lainnya,catatan_amar,tanggal_musyawarah,tanggal_dibacakan,kaidah,abstrak,nama_file,url_file) VALUES (%s, %s,%s, %s,%s, %s,%s, %s, %s,%s, %s,%s, %s,%s, %s,%s, %s, %s,%s,%s)"

        mycursor.execute(sql_insert_data_crawling_detail, values_kolom)
        mydb.commit()

        sql = "UPDATE crawling_link SET status = 'success' WHERE id = '"+str(id_row)+"'"
        mycursor.execute(sql)
        mydb.commit()
        return "success"
    except:
        sql = "UPDATE crawling_link SET status = 'error' WHERE id = '"+str(id_row)+"'"
        mycursor.execute(sql)
        mydb.commit()
        return "error"

In [14]:
mycursor.execute("SELECT * FROM crawling_link where status='belum'")
myresult = mycursor.fetchall()
total = len(myresult)
per_page = 1
for per_my_result in myresult:
    id_row = per_my_result[0]
    url_page = per_my_result[1]
    print("Processing ("+str(per_page)+"/"+str(total)+") | Message: "+extract_page_detail(id_row,url_page)+" | Url:"+url_page)
    per_page+=1
    # break

Processing (1/20) | Message: error | Url:https://putusan3.mahkamahagung.go.id/direktori/putusan/c9c2b6844bb6b085bad202671daed564.html
Processing (2/20) | Message: success | Url:https://putusan3.mahkamahagung.go.id/direktori/putusan/4e94bb5181db32b006ed0877dfb1f97a.html
Processing (3/20) | Message: success | Url:https://putusan3.mahkamahagung.go.id/direktori/putusan/26f6a0439c762929fee147cbd765aa78.html
Processing (4/20) | Message: success | Url:https://putusan3.mahkamahagung.go.id/direktori/putusan/475fed668b3e59a2af97dbffa8c8ccaa.html
Processing (5/20) | Message: success | Url:https://putusan3.mahkamahagung.go.id/direktori/putusan/e04a8e983fd65c2e9330a9356da7a8ce.html
Processing (6/20) | Message: success | Url:https://putusan3.mahkamahagung.go.id/direktori/putusan/ecec61ccb066a5434b59157ccc37ad07.html
Processing (7/20) | Message: error | Url:https://putusan3.mahkamahagung.go.id/direktori/putusan/30104370baa585c8966622b9636c8782.html
Processing (8/20) | Message: success | Url:https://p

## Download File Function

In [42]:
import wget
def download_this_file(id_row,url_name,file_name):
    nama_file = file_name.replace("/","--")
    try:
        wget.download(url_name, "file_data_download/"+nama_file)
        sql = "UPDATE crawling_detail SET status_download = 'success', crawling_file = '"+nama_file+"' WHERE id = '"+str(id_row)+"'"
        mycursor.execute(sql)
        mydb.commit()
        return nama_file,"success"
    except:
        sql = "UPDATE crawling_detail SET status_download = 'gagal' WHERE id = '"+str(id_row)+"'"
        mycursor.execute(sql)
        mydb.commit()
        return nama_file,"gagal"
    
# url_file = 'https://putusan3.mahkamahagung.go.id/direktori/download_file/3a8649a000396c9a018939d8369a500b/pdf/zaec1145e94f6220b926313531343130'

In [43]:
mycursor.execute("SELECT * FROM crawling_detail where status_download='belum'")
myresult = mycursor.fetchall()
total = len(myresult)
print(total)

17


In [45]:
per_page = 1
for _ in myresult:
    id_row = _[0]
    file_name = _[-5]
    url_name = _[-4]
    nama_file,message_hasil = download_this_file(id_row,url_name,file_name)
    print("Processing ("+str(per_page)+"/"+str(total)+") | Message: "+message_hasil+" | Nama File:"+nama_file)
    # break
    # print(_)

Processing (1/17) | Message: success | Nama File:33-K--PM.II-09--AD--III--2006.pdf
